# 📘 Análise do ENADE 2023
Este notebook realiza uma análise exploratória dos dados do ENADE com foco em: leitura de dados, data wrangling, estatísticas descritivas, agregações e visualizações para posterior exportação ao Power BI.

In [ ]:
# 📚 Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('pastel')


In [ ]:
# 📥 Leitura do arquivo
df = pd.read_excel("conceito_enade_2023.xlsx")
print("✅ Dados carregados com sucesso!")

In [ ]:
# 🧾 Informações gerais
print("📄 Info:")
print(df.info())
print("\n📊 Estatísticas descritivas:")
print(df.describe(include='all'))

In [ ]:
# 🧹 Renomeando e limpando colunas
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('*', '', regex=False).str.replace('**', '', regex=False)
print("🧾 Colunas normalizadas:")
print(df.columns.tolist())

In [ ]:
# 🎯 Seleção de colunas relevantes
colunas_relevantes = [
    'ano', 'área_de_avaliação', 'grau_acadêmico', 'nome_da_ies', 'organização_acadêmica',
    'categoria_administrativa', 'município_do_curso', 'sigla_da_uf',
    'nº__de_concluintes_participantes', 'nota_padronizada_-_fg', 'nota_padronizada_-_ce',
    'conceito_enade_(contínuo)', 'conceito_enade_(faixa)'
]
df = df[colunas_relevantes]

In [ ]:
# 🔍 Verificando e tratando valores faltantes
print("🔍 Valores nulos antes do tratamento:")
print(df.isnull().sum())

# Remover linhas sem nota
df = df.dropna(subset=['nota_padronizada_-_fg', 'nota_padronizada_-_ce'])

# Preenchendo faltantes com média
df['conceito_enade_(contínuo)'].fillna(df['conceito_enade_(contínuo)'].mean(), inplace=True)

# Drop em demais faltantes
df.dropna(inplace=True)
print("✅ Após tratamento:")
print(df.isnull().sum())

In [ ]:
# 📊 Agregações
# Média de notas por ano
media_por_ano = df.groupby('ano')[['nota_padronizada_-_fg', 'nota_padronizada_-_ce']].mean()
print("📈 Média por ano:")
print(media_por_ano)

# Média por UF
media_por_estado = df.groupby('sigla_da_uf')['conceito_enade_(contínuo)'].mean().sort_values(ascending=False)

# Média por área
media_por_area = df.groupby('área_de_avaliação')['nota_padronizada_-_fg'].mean().sort_values(ascending=False)


In [ ]:
# 📈 Visualização 1: Evolução das notas por ano
media_por_ano.plot(kind='line', marker='o', figsize=(10, 6))
plt.title('Evolução das Notas Padronizadas - FG e CE')
plt.ylabel('Nota Padronizada')
plt.grid(True)
plt.show()

In [ ]:
# 📊 Visualização 2: Conceito Enade médio por estado
plt.figure(figsize=(12, 6))
sns.barplot(x=media_por_estado.index, y=media_por_estado.values)
plt.xticks(rotation=90)
plt.title('Conceito Enade Médio por Estado (UF)')
plt.ylabel('Conceito Médio')
plt.show()

In [ ]:
# 📚 Visualização 3: Boxplot por grau acadêmico
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='grau_acadêmico', y='nota_padronizada_-_fg')
plt.title('Distribuição da Nota Padronizada - FG por Grau Acadêmico')
plt.show()

In [ ]:
# 🧭 Visualização 4: Heatmap de notas por UF e Grau Acadêmico
pivot_heatmap = df.pivot_table(values='nota_padronizada_-_fg', index='sigla_da_uf', columns='grau_acadêmico', aggfunc='mean')
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_heatmap, annot=True, fmt=".1f", cmap="YlGnBu")
plt.title('Média da Nota Padronizada - FG por UF e Grau Acadêmico')
plt.show()

In [ ]:
# 💾 Exportação para Power BI
df.to_csv('enade_tratado.csv', index=False)
print("✅ Dados exportados para 'enade_tratado.csv'. Pronto para Power BI.")